In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
237,"Being a huge die-hard Monkey Island fan, I fug...",positive
40,"Thanks Jymn Magon, for creating Disney's 2 bes...",positive
974,This movie is about a very delicate argument a...,positive
179,"-may contain spoilers-<br /><br />Clearly, who...",negative
883,In Holland a gay writer Gerard (Jeroen Krabbe)...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
237,huge die hard monkey island fan fugured one wo...,positive
40,thanks jymn magon creating disney s best carto...,positive
974,movie delicate argument searching something ma...,positive
179,may contain spoiler br br clearly ever made fi...,negative
883,holland gay writer gerard jeroen krabbe give l...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
positive    251
negative    249
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
237,huge die hard monkey island fan fugured one wo...,1
40,thanks jymn magon creating disney s best carto...,1
974,movie delicate argument searching something ma...,1
179,may contain spoiler br br clearly ever made fi...,0
883,holland gay writer gerard jeroen krabbe give l...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Roshanrajmahato/Capstone-Project.mlflow')
dagshub.init(repo_owner='Roshanrajmahato', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression experiment ")


Accessing as Roshanrajmahato

Initialized MLflow to track repo "Roshanrajmahato/Capstone-Project"

Repository Roshanrajmahato/Capstone-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/26d127bcac384ee7888f1b67e979e66c', creation_time=1750334848039, experiment_id='1', last_update_time=1750334848039, lifecycle_stage='active', name='Logistic Regression experiment ', tags={}>

In [12]:
# import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")


        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-06-20 12:05:51,206 - INFO - Starting MLflow run...
2025-06-20 12:05:51,987 - INFO - Logging preprocessing parameters...
2025-06-20 12:05:53,614 - INFO - Initializing Logistic Regression model...
2025-06-20 12:05:53,615 - INFO - Fitting the model...
2025-06-20 12:05:53,668 - INFO - Model training complete.
2025-06-20 12:05:53,670 - INFO - Logging model parameters...
2025-06-20 12:05:54,719 - INFO - Making predictions...
2025-06-20 12:05:54,721 - INFO - Calculating evaluation metrics...
2025-06-20 12:05:54,742 - INFO - Logging evaluation metrics...
2025-06-20 12:05:58,386 - INFO - Saving and logging the model...
2025-06-20 12:06:20,618 - INFO - Model training and logging completed in 28.63 seconds.
2025-06-20 12:06:20,620 - INFO - Accuracy: 0.64
2025-06-20 12:06:20,621 - INFO - Precision: 0.5757575757575758
2025-06-20 12:06:20,622 - INFO - Recall: 0.6909090909090909
2025-06-20 12:06:20,624 - INFO - F1 Score: 0.628099173553719
